This is an introduction to a the Semi-supervised Learning approach.

That situation where we have small amounts of labeled data, and we need to train our model with it, so in order to achieve a better and more robust model, we look for more information let's say images, but we need to label each image in order to make this new information meaningful for our model, this is effective for those cases where the amount of data is small, but for large amounts of data it becomes extremely tedious and time and resources consuming process. Here is were semi-supervised learning (SSL for simplicity) approach becomes relevant; the unlabeled data is used to create a more accurate boundary between the positive and negative data,  without the painfull process of manually labeling each entry on the dataset one by one.

Self training is the same as pseudo-label training?? MAYBE
XGBRegressor
There's some different approaches under  

making use of the supervised approach on the already labeled data to train the model, and it's output to label the remaining unlabeled data.


Some advantages of this approach are:

+ Less cost associated with time spent in labeling process and resources to achieve that, in terms of expertise, tools and computational power.
+ More unlabeled data, inhibit the biased learning, which improves accuracy during training process.



PSEUDO-LABELING

Making use of the supervised approach on the already labeled data to train the model, and it's output to label the remaining unlabeled data.

1) train the algorithm with the labeled portion of the data set.

2) Label the unlabeled data with the output of predictions done by the systems.

For this example a data set from Kaggle's competittion Titanic: Machine Learning from Disaster has been used, and can be downloaded from this link: https://www.kaggle.com/c/titanic/data

The purpose of that competittion was to predict values on Survived column for each PassengerId in test dataset.

In [422]:
#step 1
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.utils import shuffle
from sklearn.linear_model import BayesianRidge, Ridge, ElasticNet

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score
from sklearn.compose import TransformedTargetRegressor
from sklearn.preprocessing import MinMaxScaler


from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor


Import the dataset, for this case we will use the Titanic dataset from Kaggle's competition

In [423]:
df_train= pd.read_csv(r'C:\Users\dengo\Downloads\titanic\train.csv')
df_test = pd.read_csv(r'C:\Users\dengo\Downloads\titanic\test.csv')

Change all NaN values for ceros.

In [424]:
df_train = df_train.fillna(0)
df_test = df_test.fillna(0)

In [425]:
df_train.head()


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,0,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,0,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,0,S


Please note that the test set has not the Survived column, and the purpose for this model is to try to predict if a passenger survived or perished one by one.

In [426]:
df_test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,0,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,0,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,0,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,0,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,0,S


The following procedure, is intended to transform the data that is originally in form of strings to a quatizable form, so it is converted to integers, for both train andd test datasets.

In [427]:
features = df_train.columns[2:]

for column_name in features:
    label_encoder = LabelEncoder() 
    
    # Obtain the column values from training and test datasets
    df_train_column_values = list(df_train[column_name].values)
    df_test_column_values = list(df_test[column_name].values)

    
    # Fit the label encoder
    label_encoder.fit(df_train_column_values + df_test_column_values)
    
    # Transform the feature
    df_train[column_name] = label_encoder.transform(df_train_column_values)
    df_test[column_name] = label_encoder.transform(df_test_column_values)

all qualitative values were transformed to numerical values.

In [428]:
df_test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,2,638,1,53,0,0,376,43,0,2
1,893,2,1266,0,70,1,0,582,12,0,3
2,894,1,837,1,87,0,0,184,77,0,2
3,895,2,1278,1,42,0,0,366,63,0,3
4,896,2,543,0,33,1,1,338,93,0,3


In [429]:
df_test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,2,638,1,53,0,0,376,43,0,2
1,893,2,1266,0,70,1,0,582,12,0,3
2,894,1,837,1,87,0,0,184,77,0,2
3,895,2,1278,1,42,0,0,366,63,0,3
4,896,2,543,0,33,1,1,338,93,0,3


Survived is the column that is missing in Test dataset, and we want to create that column for that dataset and predict survivors by passenger ID 

In [430]:
target = 'Survived'
X_train, X_test = df_train[features], df_test[features]
y_train = df_train[target]


The next class is in charge of creating/assigning the labels, create a new dataset with the pseudo labeled information and predict information based on the target 

In [431]:
class PseudoLabeler(BaseEstimator, RegressorMixin):
    '''
    Sci-kit learn wrapper for creating pseudo-lebeled estimators.
    '''
    
    def __init__(self, model, unlabled_data, features, target, sample_rate=0.5, seed=42):
        '''
        @sample_rate - percent of samples used as pseudo-labelled data
                       from the unlabled dataset
        '''
        assert sample_rate <= 1.0, 'Sample_rate should be between 0.0 and 1.0.'
        
        self.sample_rate = sample_rate
        self.seed = seed
        self.model = model
        self.model.seed = seed
        
        self.unlabled_data = unlabled_data
        self.features = features
        self.target = target
        
    def get_params(self, deep=True):
        return {
            "sample_rate": self.sample_rate,
            "seed": self.seed,
            "model": self.model,
            "unlabled_data": self.unlabled_data,
            "features": self.features,
            "target": self.target
        }

    def set_params(self, **parameters):
        for parameter, value in parameters.items():
            setattr(self, parameter, value)
        return self

        
    def fit(self, X, y):
        '''
        Fit the data using pseudo labeling.
        '''
        augemented_train = self.__create_augmented_train(X, y)
        self.model.fit(augemented_train[self.features],augemented_train[self.target])
        
        return self


    def __create_augmented_train(self, X, y):
        '''
        Create and return the augmented_train set that consists
        of pseudo-labeled and labeled data.
        '''        
        num_of_samples = int(len(self.unlabled_data) * self.sample_rate)
        
        # Train the model and creat the pseudo-labels
        self.model.fit(X, y)
        pseudo_labels = self.model.predict(self.unlabled_data[self.features])
        
        # Add the pseudo-labels to the test set
        pseudo_data = self.unlabled_data.copy(deep=True)
        pseudo_data[self.target] = pseudo_labels
        
        # Take a subset of the test set with pseudoa-labels and append in onto
        # the training seta
        sampled_pseudo_data = pseudo_data.sample(n=num_of_samples)
        temp_train = pd.concat([X, y], axis=1)
        augemented_train = pd.concat([sampled_pseudo_data, temp_train])
        return augemented_train, pseudo_data
       
            
    def predict(self, X):
        '''
        Returns the predicted values.
        '''
        return self.model.predict(X)
    
    def get_model_name(self):
        return self.model.__class__.__name__

In [432]:
model= PseudoLabeler (
        GradientBoostingRegressor(),
        df_test,
        features,
        target, 
        sample_rate = 0.5
    )


model.fit(X_train, y_train)
prediction = model.predict(X_test)

scores = cross_val_score(model, X_train, y_train, scoring='r2', n_jobs=8)
score_description = "R2: %0.4f (+/- %0.4f)" % (scores.mean(), scores.std() * 2)
print('{model:25}  {score_cv}'.format(
    model=model.__class__.__name__,
   # num_folds=num_folds,
    score_cv=score_description
    ))


C:\Users\dengo\Miniconda3\lib\site-packages\ipykernel_launcher.py:67: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



TypeError: tuple indices must be integers or slices, not Index

Assuming values above 0.41 as survived (1), a simple manipulation has been done to the data in order to be able to visualize it better, but a better understanding of the values obtained is necessary to obtain better results.

In [433]:
pred_length = len(prediction)

for i in range(pred_length):
    if prediction[i] >= 0.42:
        prediction[i] = 1
    else: prediction[i] = 0
        


Save the result

In [434]:
result = pd.DataFrame(data = prediction, columns = [ 'Survived'])
result.insert(0, column='PassengerId', value=df_test['PassengerId'])

result.to_csv('pseudo-labelling.csv', index='False')


In [435]:
result

,PassengerId,Survived
0,892,0.0
1,893,0.0
2,894,0.0
3,895,0.0
4,896,1.0
5,897,0.0
6,898,1.0
7,899,0.0
8,900,1.0
9,901,0.0
